IPython 7.0 introduced the ability to use the `await` statement in code cells.  This post riffs off of that and considers a worl where the `return` statement were valid in code cells.

In [1]:
    import ast, asyncio, pytest
    ip = get_ipython()

With the old behavior `ip.autoawait = False`

In [2]:
    ip.autoawait = False

an `await` statement will raise a `SyntaxError`.

In [3]:
    assert isinstance(ip.run_cell('await asyncio.sleep(0)').error_before_exec, SyntaxError)

SyntaxError: invalid syntax (<ipython-input-3-51b56ae34a91>, line 1)

We reset the original conditions

In [4]:
    ip.autoawait = True

and the `await` statement is valid.

In [5]:
    await asyncio.sleep(0)

Let's try the same idea with `return`

In [6]:
    assert isinstance(ip.run_cell('return 11').error_before_exec, SyntaxError)

SyntaxError: 'return' outside function (<ipython-input-6-3542c47c7e35>, line 1)

when `return` is encountered at the parent level of the cell we will call `IPython.display.display` instead.

In [7]:
    import ast
    class ReturnDisplay(ast.NodeTransformer):
        def visit_FunctionDef(self, node): return node
        visit_ClassDef = visit_FunctionDef
        def visit_Return(self, node): return ast.Expr(
            ast.Call(
                func=ast.parse('__import__("IPython").display.display', mode='eval').body, 
                args=node.value.elts if isinstance(node.value, ast.Tuple) else [node.value], 
                keywords=[]))

In [8]:
    def load_ipython_extension(ip): ip.ast_transformers.append(ReturnDisplay())
    def unload_ipython_extension(ip): ip.ast_transformers = [x for x in ip.ast_transformers if not isinstance(x, ReturnDisplay)]

Register the node transformer.

In [9]:
    if __name__ == "__main__": load_ipython_extension(get_ipython())

Demonstrate that the transformer works.

In [10]:
        x = 11
        return x

11

The reason for the modification is that the distance between and interactive code and reusable functions is shortened.

In [11]:
    def f(x = 11):
        return x

It also provides a visual cue for the goal of a function,